In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.6.3'

# Import Scrapy

In [2]:
try:
    import scrapy
    #from scrapy_splash import SplashRequest
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

# Setting up a pipe line

In [3]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('result.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

# Creating the spider

In [4]:
import logging
from scrapy.utils.markup import remove_tags
from scrapy.spiders import CrawlSpider, Rule
from w3lib.html import replace_escape_chars
from scrapy.selector import HtmlXPathSelector
from scrapy.linkextractors import LinkExtractor
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

class jobSpider(scrapy.Spider):
    name = "job"
    
    start_urls = ['http://jobs.bdjobs.com/jobdetails.asp?id=793245&fcatId=1&ln=1']
    
    for i in range(780000 , 789999):
        start_urls.append('http://jobs.bdjobs.com/jobdetails.asp?id=' + str(i))
    
    for i in range(790000 , 799999):
        start_urls.append('http://jobs.bdjobs.com/jobdetails.asp?id=' + str(i))
    
        
    
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'result.txt'                        # Used for pipeline 2
    }
    

    
    def parse(self, response):
        
        #links = response.css('div.job-title-text a::attr(href)').extract()
        para = response.xpath('//p')
        uls =  response.xpath('//ul')
        h4s =  response.xpath('//h4')
        
        if len(para) == 5 and len(uls) > 16 and len(h4s) > 7: 
            
            yield {   
            
            'Title': remove_tags(" ".join(response.xpath('//h4/text()').extract_first().split())),
                
            'Category': remove_tags(" ".join(response.xpath("//p/strong/following-sibling::text()").extract_first().split())),
            
            'Company Name': remove_tags(" ".join(response.xpath('//h2/text()').extract_first().split())),
            
            'Vacancy': remove_tags(" ".join(para[1].extract().split())),
                
            'Job Responsibility': remove_tags(" ".join(response.xpath("//div[@class='job_des']/h5/following-sibling::ul/li/text()").extract_first().split())),
                
            'Employment Status': remove_tags(" ".join(para[2].extract().split())),
                
            'Educational Requirement': remove_tags(" ".join(response.xpath("//div[@class='edu_req']/h5/following-sibling::ul/li/text()").extract_first().split())),
                
            'Additional Requirement': remove_tags(" ".join(response.xpath("//div[@class='job_req']/h5/following-sibling::ul/li/text()").extract_first().split())),
                
            'Job Location': remove_tags(" ".join(para[3].extract().split())),
                
            'Salary': remove_tags(" ".join(response.xpath("//div[@class='salary_range']/h5/following-sibling::ul/text()").extract_first().split())),
            
            'Job Source': remove_tags(" ".join(para[4].extract().split())),
            
            'Application Deadline': remove_tags(" ".join(response.xpath("//span[@class='date']/text()/following-sibling::strong/text()").extract_first().split())),  
            }

class linkSpider(scrapy.Spider):
    
    name = "links"
    
    start_urls = ['http://jobs.bdjobs.com/jobsearch.asp',]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'links.txt'                        # Used for pipeline 2
    }
   
    
    def parse(self, response):
        
        yield {
            'url' : response.css('div.job-title-text a::attr(href)').extract()
        }
        next_page = response.css('li.prevnext a::attr(href)').extract_first()
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)
        
 
        

        
        

# Start Crawler

In [5]:

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(jobSpider)
process.start()
#process.end()

2018-09-27 00:33:46 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-09-27 00:33:46 [scrapy.utils.log] INFO: Versions: lxml 4.1.0.0, libxml2 2.9.4, cssselect 1.0.3, parsel 1.5.0, w3lib 1.19.0, Twisted 18.7.0, Python 3.6.3 |Anaconda, Inc.| (default, Oct 13 2017, 12:02:49) - [GCC 7.2.0], pyOpenSSL 17.2.0 (OpenSSL 1.0.2l  25 May 2017), cryptography 2.0.3, Platform Linux-4.10.0-38-generic-x86_64-with-debian-stretch-sid
2018-09-27 00:33:46 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'result.txt', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


2018-09-27 00:48:25 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=785029> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

2018-09-27 00:53:01 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=786997> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

2018-09-27 00:53:31 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=787145> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

2018-09-27 00:54:04 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=787395> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

...

2018-09-27 01:09:56 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=793108> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

2018-09-27 01:10:05 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=793173> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

2018-09-27 01:10:14 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=793233> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

2018-09-27 01:10:24 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=793303> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython

2018-09-27 01:10:29 [scrapy.core.scraper] ERROR: Spider error processing <GET http://jobs.bdjobs.com/jobdetails.asp?id=793341> (referer: None)
Traceback (most recent call last):
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 102, in iter_errback
    yield next(it)
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/offsite.py", line 30, in process_spider_output
    for x in result:
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/referer.py", line 339, in <genexpr>
    return (_set_referer(r) for r in result or ())
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/urllength.py", line 37, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "/home/tensor_cs/anaconda3/lib/python3.6/site-packages/scrapy/spidermiddlewares/depth.py", line 58, in <genexpr>
    return (r for r in result or () if _filter(r))
  File "<ipython